<a href="https://colab.research.google.com/github/Markkolas/TP2/blob/jorge-test/YoloV5_Dataset_Completo/YoloV5_Dataset1Clase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Re-entrenamos el modelo medium de YoloV5 con el dataset entero de Kaggle

Añadimos acceso a la carpeta de los dataset del Drive del grupo de proyectos

Importante estar ya logeados en la cuenta del grupo, da problemas si estamos en nuestra cuenta personal y queremos montar el drive de la cuenta del grupo.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importamos las bibliotecas necesarias

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
# Para windows con la CPU:
!pip3 install torch torchvision torchaudio 

# Para Windows con CUDA:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

# Para windows con la CPU:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# Para Windows con CUDA:
# !pip3 install torch torchvision torchaudio

# Es posible que la mayoría de dependencias ya esten instaladas ^^

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone https://github.com/ultralytics/yolov5
# Se deberia haber creado una carpeta en nuestro area de trabajo
# Si ya lo teniamos saldra el siguiente error =>fatal: destination path 'yolov5' already exists and is not an empty directory.

Cloning into 'yolov5'...
remote: Enumerating objects: 15543, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 15543 (delta 60), reused 93 (delta 45), pack-reused 15393
Receiving objects: 100% (15543/15543), 14.59 MiB | 18.20 MiB/s, done.
Resolving deltas: 100% (10584/10584), done.


In [ ]:
%cd /content/yolov5
!git pull
%cd ../

/content/yolov5
Already up to date.
/content


In [ ]:
!pip install -r yolov5/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00


In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2 #Para abrir webcam
# cv2.imshow() is disabled in Colab, because it causes Jupyter sessionsto crash, consider using cv2_imshow 
from google.colab.patches import cv2_imshow 
import sys # to access the system

# Esta línea de código establece una variable de entorno llamada KMP_DUPLICATE_LIB_OK con el valor "True", lo que significa que permite la carga de bibliotecas duplicadas en el entorno de Python. Esto es útil para algunas bibliotecas de Machine Learning que usan OpenMP para acelerar el cómputo.
import os    
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

***
En la siguiente celda se escogen 100 imagenes para hacer el re-entreno.
 
Además esas imagenes se suben a unas carpetas que creamos en el directorio base de colabs.
***

In [ ]:
import os
import random
import shutil

# Creamos los directorios necesarios
os.makedirs('/content/data/train/images', exist_ok=True)
os.makedirs('/content/data/train/labels', exist_ok=True)
os.makedirs('/content/data/val/images', exist_ok=True)
os.makedirs('/content/data/val/labels', exist_ok=True)

dest_train_Image = "/content/data/train/images/"
dest_train_Label = "/content/data/train/labels/"
dest_val_Image = "/content/data/val/images/"
dest_val_Label = "/content/data/val/labels/"


with os.scandir('/content/drive/MyDrive/IA/Datasets/Kaggle1Clase/train/images') as entries:
  for entry in entries:
    shutil.copy2(entry.path, dest_train_Image)

with os.scandir('/content/drive/MyDrive/IA/Datasets/Kaggle1Clase/train/labels') as entries:
  for entry in entries:
    shutil.copy2(entry.path, dest_train_Label)

with os.scandir('/content/drive/MyDrive/IA/Datasets/Kaggle1Clase/val/images') as entries:
  for entry in entries:
    shutil.copy2(entry.path, dest_val_Image)

with os.scandir('/content/drive/MyDrive/IA/Datasets/Kaggle1Clase/val/labels') as entries:
  for entry in entries:
    shutil.copy2(entry.path, dest_val_Label)


with os.scandir('/content/drive/MyDrive/IA/Datasets/Plastico1Clase/train/images') as entries:
  for entry in entries:
    shutil.copy2(entry.path, dest_train_Image)

with os.scandir('/content/drive/MyDrive/IA/Datasets/Plastico1Clase/train/labels') as entries:
  for entry in entries:
    shutil.copy2(entry.path, dest_train_Label)

with os.scandir('/content/drive/MyDrive/IA/Datasets/Plastico1Clase/val/images') as entries:
  for entry in entries:
    shutil.copy2(entry.path, dest_val_Image)

with os.scandir('/content/drive/MyDrive/IA/Datasets/Plastico1Clase/val/labels') as entries:
  for entry in entries:
    shutil.copy2(entry.path, dest_val_Label)

***
**`Entrenar un modelo desde cero`**
***

Ahora tenemos que crear el archivo dataset.yaml, que contiene las categorias de nuestros pesos y las rutas de nuestras imagenes de entrenamiento, test y validación.

In [ ]:
f= open("yolov5/data/Dataset1Clase.yaml","w+")

f.write("path: ../data \ntrain: train/images/ \nval: val/images \ntest: test/images \n\nnc: 1 \n\nnames: ['Señal']")

f.close()

In [ ]:
shutil.copy2("/content/yolov5/requirements.txt", "/content/requirements.txt")
!cd yolov5 && python train.py --img 640 --batch 16 --epochs 300 --data Kaggle1Clase.yaml --weights yolov5s.pt --project ../drive/MyDrive/IA/Pesos_yoloV5/Dataset1Clase --save-period 20 

usage: train.py
       [-h]
       [--weights WEIGHTS]
       [--cfg CFG]
       [--data DATA]
       [--hyp HYP]
       [--epochs EPOCHS]
       [--batch-size BATCH_SIZE]
       [--imgsz IMGSZ]
       [--rect]
       [--resume [RESUME]]
       [--nosave]
       [--noval]
       [--noautoanchor]
       [--noplots]
       [--evolve [EVOLVE]]
       [--bucket BUCKET]
       [--cache [CACHE]]
       [--image-weights]
       [--device DEVICE]
       [--multi-scale]
       [--single-cls]
       [--optimizer {SGD,Adam,AdamW}]
       [--sync-bn]
       [--workers WORKERS]
       [--project PROJECT]
       [--name NAME]
       [--exist-ok]
       [--quad]
       [--cos-lr]
       [--label-smoothing LABEL_SMOOTHING]
       [--patience PATIENCE]
       [--freeze FREEZE [FREEZE ...]]
       [--save-period SAVE_PERIOD]
       [--seed SEED]
       [--local_rank LOCAL_RANK]
       [--entity ENTITY]
       [--upload_dataset [UPLOAD_DATASET]]
       [--bbox_interval BBOX_INTERVAL]
       [--artifact_a

In [ ]:
# !cd yolov5 && python train.py --resume ../drive/MyDrive/IA/Pesos_yoloV5/Dataset1Clase/exp/weights/last.pt